<a href="https://colab.research.google.com/github/xuxoramos/nosql-4-ds/blob/gh-pages/tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro a TF/IDF

En este tutorial veremos una introducción a Google Colab y a TF/IDF.

Google Colab es un server de Jupyter Notebooks _on the fly_ que se aventaron los cuates de _Big G_, y que además trae una bola de cosas interesantes como GPUs on demand (con costo) y otras cositas.

Con los notebooks habilitamos un paradigma que se llama _literate programming_, que combina texto, figuras y gráficas con **el código que las genera**.

Esto ha representado un gran avance para mitigar la crisis de reproducibilidad de la que hemos hablado, porque podemos entonces escribir un artículo o un paper para journal donde código, datos, gráficas y argumentos **están en el mismo lugar**.

Mejor aún, si luego hacemos este trabajo público a través de Github, entonces estaremos haciendo **verdadero open science**.

Esto obviamente trae otros problemas, como falta de verificación o poca formalidad en el mecanismo de arbitraje, pero como todas las tendencias, hemos estado bajo el yugo del arbitraje tanto tiempo, y los mecanismos de producción científica están tan anquilosados y calcificados, que este _swing_ tan fuerte a un paradigma más abiertos va a ponernos en un punto medio durante algo de tiempo.

Vamos a desarrollar un ejemplo de TF/IDF para ver como esto nos ayuda.

## Intuición detrás de TF/IDF

Esta técnica analiza los términos y palaras en un _corpus_ (i.e. una colección de documentos, sin estructura, de cualquier longitud u origen) y muestra aquellos que ocurren menos entre todos ellos.

Por ejemplo, en un _corpus_ hecho de puras recetas de cocina, los verbos "batir", "picar", "moler", o los sustantivos "masa", "cilantro", "bicarbonato" no son términos sobresalientes, pero si por alguna razón se especificara que cierta receta se debe cocinar en "cast iron" o en "horno holandés", y de entre todas las recetas, solamente 1 o 2 tuvieran estas instrucciones, entonces el TF/IDF resaltaría justo estas palabras.

Para poder hacer este análisis vamos a atravesar los mismos pasos que cualquier análisis de datos:

1. Adquisición de datos (nos vamos a basar en [el videito donde nos conectamos al APILegislativo](https://drive.google.com/file/d/1-syp_3A1cD-kOOAUhP1Bl_Cox6m9QxxU/view?usp=sharing)
2. Limpieza/preparación de datos
3. Análisis exploratorio (aquí vamos a hacer el TF/IDF)
4. Modelado (aquí desarrollaríamos un clasificador usando algoritmos supervisados, pero no haremos esto)
5. Desarrollo de API productivo (esto tampoco lo vamos a hacer, pero implica **exponer** el modelo a través de una API para poder hacer clasificaciones de texto en producción que luego podemos integrar a una aplicacion móvil o web).

In [2]:
# Adquisición de datos
# Estos datos ya los adquirí usando mi propio token obtenido con lo que vimos en el video donde presentamos el API Legislativo
# Vamos a ponerlos en un array de Python
docs = ["propone modificar 'esposo o concubino' por el vocablo 'cónyuge'; y se contempla la posibilidad de otorgarle a la mujer trabajadora, el derecho de transmitir al esposo o al concubinario una pensión de viudez en el caso eventual de su fallecimiento.adiciona la definición de 'unión civil', entendiéndola como el acto jurídico bilateral que se constituye cuando las personas físicas de diferente o del mismo sexo, mayores de edad y con capacidad jurídica plena, establecen un hogar común con voluntad de permanencia y ayuda mutua, por el que se deriven obligaciones alimentarias, de sucesión o semejantes y que esté reconocido en la legislación de los estados, cualquiera que sea la denominación que adquiera.",
"propone establecer como obligación del estado-federación, la impartición de la educación superior.",
"la iniciativa tiene como propósito reformar diversos artículos de la ley federal de presupuesto y responsabilidad hacendaria, con el objetivo de eliminar los privilegios de la alta burocracia federal y establecer un amplio planteamiento de austeridad republicana.",
"se propone la desaparición de la comisión nacional de los salarios mínimos y la creación de un organismo constitucional autónomo del estado, denominado instituto nacional del salario digno, que tendrá a su cargo realizar los estudios para determinar los salarios dignos general y profesionales, bajo los principios de acceso a la información, transparencia, objetividad, independencia y compromiso social. en la iniciativa se propone que los salarios dignos sean anuales, pero el instituto podrá acordar un ajuste en algún otro momento del año, en función del deterioro salarial sufrido por el incremento del costo de la vida.",
"se propone que en el proyecto del presupuesto de egresos de cada año, deberá considerarse el incremento del salario mínimo vigente en el país, cualquiera que sea su denominación, a efecto de que sea incrementado dos puntos porcentuales por encima de la proyección inflacionaria del año que inicie; o bien, de acuerdo con la estimación más reciente, emitidas éstas invariablemente por el banco de méxico.",
"propone establecer en el artículo 50 constitucional que el funcionamiento del poder legislativo federal se regirá por los principios de eficiencia, austeridad y participación pública.",
"la iniciativa tiene como propósito modificar el artículo 3 de la constitución política de los estados unidos mexicanos, con el objetivo de garantizar el derecho y el acceso universal a la educación superior de calidad.",
"la iniciativa tiene como propósito adicionar un párrafo al artículo 83 de la constitución política de los estados unidos mexicanos, con el objetivo de eliminar las pensiones de retiro que reciben los expresidentes, así como los beneficios en materia social y administrativa.",
"tiene por objeto establecer el permiso laboral para impulsar la participación activa de los padres de familia en la educación de sus hijos, colaborando con las instituciones escolares al menos una vez al mes, en actividades para el mejoramiento educativo.",
"se busca agregar la obligatoriedad de presentar el dictamen de congruencia patrimonial elaborado por contador público registrado en los términos de ley, el cual servirá de base para la investigación o auditoría y también para solicitar las aclaraciones al declarante, a efecto de iniciar los procedimientos aplicables.se propone que con base en el dictamen de congruencia patrimonial, en relación con la declaración inicial, declaración de conflicto de interés y declaración fiscal, la revisión que realice de ésta las autoridades, cuando el sujeto a la verificación de la evolución de su patrimonio no justifique la procedencia lícita del incremento notoriamente desproporcionado de éste, tendrá como consecuencia la sanción, es decir, la destitución de su empleo, cargo o comisión.",
"propone paridad de género en los tres poderes y en los tres órdenes de gobierno, esto es, que la mitad de los órganos y/o instituciones sea compuesto por mujeres. en lo que corresponde al poder ejecutivo a su gabinete, en el legislativo a las y los diputados y a las senadoras y senadores del h. congreso de la unión. en el judicial a las y los ministros, a las y los jueces de distrito y magistradas y magistrados de circuito y electorales, así como al consejo de la judicatura federal. en los organismos públicos autónomos se refiere a los órganos de dirección del ine, inai, instituto nacional para la evaluación de la educación, instituto nacional de estadística y geografía, comisión federal de competencia económica, comisión nacional de derechos humanos y al instituto nacional de telecomunicaciones. igualmente, se refiere al tribunal de justicia administrativa y al tribunal superior agrario.",
"propone abrogar la ley de seguridad interior, en razón de que el congreso no tiene facultades para legislar en la materia; violenta el artículo 1 constitucional al ser contraria a las normas de derechos humanos contenidas en las convenciones internacionales que han sido signadas por el estado mexicano, y violenta diversos artículos constitucionales.",
"la iniciativa tiene como propósito eliminar, adicionar y corregir diversos apartados de la ley general de responsabilidades administrativas, con el objetivo de evitar la impunidad de las conductas administrativas, garantizando que los eventuales infractores no queden sin sanción y las conductas ilegales queden claramente tipificadas.",
"contempla incorporar un párrafo segundo al artículo 44 bis 4 de la ley de instituciones de crédito a efecto de señalar que las instituciones de banca de desarrollo, los fideicomisos públicos constituidos por el gobierno federal para el fomento económico que realicen actividades financieras, y la financiera nacional de desarrollo agropecuario, rural, forestal y pesquero, deberán destinar al menos 20 por ciento de sus recursos a programas, productos o servicios financieros para que atiendan las necesidades específicas de las mujeres en materia de ahorro, inversión, crédito y mecanismos de protección.",
"la ley que se pretende expedir tiene por objeto reglamentar los artículos 74 fracción iv, 75, 126, 127 y 134 de la constitución política de los estados unidos mexicanos, en materia de programación, presupuestación, aprobación, ejercicio, control y evaluación de los ingresos y egresos públicos federales.los sujetos obligados a cumplir las disposiciones de la ley deberán observar que la administración de los recursos públicos federales se realice con base en criterios de legalidad, honestidad, eficiencia, eficacia, economía, racionalidad, austeridad, transparencia, control, rendición de cuentas, equidad de género, respeto a los derechos humanos e interés superior de la niñez.",
"propone establecer que el estado garantizará a toda persona, el derecho al acceso, disposición y saneamiento de agua para consumo personal y doméstico en forma suficiente, salubre, aceptable y asequible. señala que el financiamiento, construcción, operación y mantenimiento de la infraestructura hidráulica serán públicos. se prohíbe la celebración de contratos con particulares así como el otorgamiento de concesiones totales o parciales para operar, conservar, mantener, rehabilitar, modernizar o ampliar la infraestructura hidráulica. tampoco se otorgarán concesiones o contratos para proyectar, construir, equipar, operar y mantener la infraestructura hidráulica o para prestar los servicios asociados a ésta. y se deroga el capítulo ii que se denomina 'participación de inversión privada y social en obras hidráulicas federales'.",
"la iniciativa tiene como propósito adicionar un artículo 54 bis a la ley de responsabilidades administrativas, con el objetivo de incorporar la conducta denominada 'simulación de reintegro de recursos' como falta grave.",
"propone reformar el código nacional de procedimientos penales para establecer que los bienes asegurados causarán abandono en partes iguales a favor de las secretarías de salud, de educación y de bienestar o de sus equivalentes en las entidades federativas, según corresponda, y no en favor la procuraduría general de la república o de las equivalentes en las entidades federativas.busca que la autoridad judicial, en caso de decomiso de bienes, el numerario decomisado y los recursos que se obtengan por la enajenación de los bienes decomisados, una vez satisfecha la reparación a la víctima, sean entregados en partes iguales al fondo previsto en la ley general de víctimas, y a las secretarías de salud, de educación y de bienestar o a sus equivalentes en las entidades federativas y que el destino de los recursos proporcionados a las secretarías se canalice a la implementación de políticas públicas y acciones integrales para la prevención social de la delincuencia, mediante la salud, educación y deporte, particularmente de jóvenes y menores de edad, así como el combate a la pobreza y las adicciones.pretende entonces, que dejen de destinarse estos recursos al poder judicial de la federación, y a la procuraduría general de la república y sus equivalentes en las entidades federativas.",
"propone que si al fallecimiento del ejidatario resultan dos o más personas con derecho a heredar, las personas con esta perspectiva de derecho, gozarán de tres meses a partir de la muerte del ejidatario para convenir y ratificar ante la autoridad jurisdiccional correspondiente quién de ellos, conservará los derechos ejidales. para el caso de que el fallecido, fuese titular de diversos derechos parcelarios, estos podrán ser asignados de manera individual o conjunta entre las personas con derecho a heredar previstas en este párrafo, de conformidad con el convenio correspondiente hecho ante el tribunal agrario.",
"propone abrogar la ley de seguridad interior, en razón de ser inconstitucional y violentar diversos preceptos."]

In [1]:
# Vamos a importar las librerías necesarias
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# La explicación técnica la van a ver en su clase de Fuentes de Datos, por lo que solo voy a darles la receta solita.
tfidf_vectorizer=TfidfVectorizer(use_idf=True)
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(docs)
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[1]
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df.sort_values(by=["tfidf"],ascending=False).head(45)

,tfidf
impartición,0.412795
obligación,0.412795
federación,0.362853
estado,0.299933
superior,0.299933
educación,0.277477
establecer,0.258490
la,0.258362
del,0.227535
como,0.192100
